In [1]:
import pandas as pd
import requests
import os 
import ast
import logging
import time
from time import sleep
import json

In [2]:
df_fmc = pd.read_csv("df_fmc.csv")
df_fmc = df_fmc[['title', 'author', 'fmc_name']]
df_sample = df_fmc.sample(n=100, random_state=26)
df_sample

,title,author,fmc_name
1325,Dirty Little Secrets JJ Graves Mystery 1,"Hart, Liliana",J.J. Graves
1649,The Tycoons Vacation Baby for the Billionaire 2,"Anne, Melody",Trinity Mathews
1372,White Girl Problems,"Walker, Babe",Babe Walker
549,The Empty House,"Pilcher, Rosamunde",Virginia Keile
659,The Unicorn,"Murdoch, Iris",Hannah Crean-Smith
...,...,...,...
2380,Charmed to Death Ophelia Abby 2,"Damsgaard, Shirley",Ophelia Jensen
1935,Dealova,"Nuranindya, Dyan",Karra
1730,The Promise Fallen Star 4,"Sorensen, Jessica",Gemma Lucas
1471,A Boy of Good Breeding,"Toews, Miriam",Knute


In [ ]:
# # Only check first two books
# df_sample = df_sample[:10]
# df_sample

,title,author,fmc_name
1325,Dirty Little Secrets JJ Graves Mystery 1,"Hart, Liliana",J.J. Graves
1649,The Tycoons Vacation Baby for the Billionaire 2,"Anne, Melody",Trinity Mathews
1372,White Girl Problems,"Walker, Babe",Babe Walker
549,The Empty House,"Pilcher, Rosamunde",Virginia Keile
659,The Unicorn,"Murdoch, Iris",Hannah Crean-Smith
245,Child of the Dark Prophecy,"Barron, T.A.",Elli
2424,Redemption Ryan Drake 1,"Jordan, Will",Maras
2132,Avoiding Responsibility Avoiding 2,"Linde, K.A.",Lexi Walsh
433,Someone I Loved Je laimais,"Gavalda, Anna",Chloé
1728,Shadows Claim Immortals After Dark 12 The Daci...,"Cole, Kresley",Bettina of Abaddon


In [3]:
system_prompt = """
    You are a helpful literary assistant. Your job is to search the internet and assess several aspects for books that are provided to you.

    Most of the questions are about a female major character of the book whose name will be provided alongside the book title. 
    For all questions about the female character, consider any occurrence of the trait or behavior at any point in the book, 
    and give a short explanation for your answer. 

    1. What's the gender of the author? (0 = male, 1 = female, 99 = unclear or non-binary) 
    2. In what year was the book published? (numeric) 
    3. (As a list) What is/are the genre(s) of the book? (list of a maximum of 5 genres in the order of their relevance, try to use common genre names, lower case only) 
    4. Is the female character saved by a male character? (0 = no, 1 = yes, 99 = no information) 
    5. Does the female character save a male character? (0 = no, 1 = yes, 99 = no information) 
    6. Is the female character protected by a male character? (0 = no, 1 = yes, 99 = no information) 
    7. Does the female character protect a male character? (0 = no, 1 = yes, 99 = no information) 
    8. Is the female character’s problem solved through help or luck? (0 = no, 1 = yes, 99 = no information) 
    9. Does the female character solve her own problem through skill? (0 = no, 1 = yes, 99 = no information) 
    10. Is the female character victimized/harmed by a male character? (0 = no, 1 = yes, 99 = no information) 
    11. Is the female character a perpetrator or does she harm a male character? (0 = no, 1 = yes, 99 = no information) 
    12. Does the female character follow orders? (0 = no, 1 = yes, 99 = no information) 
    13. Does the female character give orders? (0 = no, 1 = yes, 99 = no information) 
    14. Is the female character physically weak/incapable? (0 = no, 1 = yes, 99 = no information) 
    15. Is the female character physically strong/capable? (0 = no, 1 = yes, 99 = no information) 
    16. Is the female character fearful/scared of taking risks/challenges? (0 = no, 1 = yes, 99 = no information) 
    17. Is the female character brave/likes taking risks/challenges? (0 = no, 1 = yes, 99 = no information) 
    18: Does the female character care for others (e.g., nurses someone, comforts someone, takes care of children or sick)? (0 = no, 1 = yes, 99 = no information)  
    19: Does the female character not engage in caregiving behaviors (e.g., focuses on tasks or goals without providing emotional or physical support to others)? (0 = no, 1 = yes, 99 = no information) 
    20: Is the female character emotionally aware and attuned to others' needs, moods, or the dynamics between people, and tries to maintain harmony? (0 = no, 1 = yes, 99 = no information) 
    21: Does the female character focus more on practical matters or impersonal concerns than others’ emotions or social dynamics? (0 = no, 1 = yes, 99 = no information) 
    22: Does the female character sacrifice her own goals, desires, or needs to prioritize someone else’s wellbeing or to avoid conflict? (0 = no, 1 = yes, 99 = no information)  
    23: Does the female character maintain personal goals and boundaries, even when others express emotional needs or expectations? (0 = no, 1 = yes, 99 = no information) 
    24. Is the female character a homemaker? (0 = no, 1 = yes, 99 = no information)   
    25. Is the female character a breadwinner? (0 = no, 1 = yes, 99 = no information) 
    26. Is the female character admired for her beauty? (0 = no, 1 = yes, 99 = no information) 
    27. Is the female character admired for her intelligence? (0 = no, 1 = yes, 99 = no information) 
    28. Does the female character have a lower rank occupation (e.g., nurse, assistant, maid, …)? (0 = no, 1 = yes, 99 = no information) 
    29. Does the female character have a higher rank occupation (doctor, manager, ruler, …)? (0 = no, 1 = yes, 99 = no information) 

    For example:
    [
    [1, 1], # first question, female author
    [2, 2011], # second question, publication year is 2011
    [3, ["mystery", "romance", "suspense"]], # third question, genres are mystery, romance, suspense
    [4, 1, "Paul saves Anne from drowning."], # fourth question, answer is yes (so 1), short explanation why 
    [5, 0, "Anne doesn't save any male character."], # fifth question, answer is no (so 0), short explanation why 
    [6, 1, "Anne is protected by her male friend."], # sixth question, answer is yes (so 1), short explanation why
    ...
    ]

"""

user_prompt_template = """
    Please answer the questions defined in your system prompt for the book {title} by {author}.
    The female character in question is {fmc_name}.
"""

In [4]:
from pydantic import RootModel
from typing import List, Literal, Tuple

class AnswerFormat(RootModel):
    root: Tuple[
        Tuple[int, int],               # Q1, author gender (e.g. [0, 1])
        Tuple[int, int],               # Q2, publication year (e.g. [1, 1998])
        Tuple[int, List[str]],         # Q3, answer, list of genres
        Tuple[int, int, str],          # Q4, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q5, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q6, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q7, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q8, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q9, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q10, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q11, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q12, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q13, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q14, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q15, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q16, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q17, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q18, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q19, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q20, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q21, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q22, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q22, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q23, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q24, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q25, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q26, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q27, yes/no/unknown, explanation
        Tuple[int, int, str],          # Q28, yes/no/unknown, explanation
        Tuple[int, int, str]           # Q29, yes/no/unknown, explanation
    ]

In [5]:
# Set up logging for error tracking
logging.basicConfig(filename='features_log.log', level=logging.INFO)

# Setting API key and headers
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# Endpoint for the API
url = "https://api.perplexity.ai/chat/completions"

# Store responses, citations, and titles + authors for progress
responses = []
citations = []
titles = []
authors = []
fmc_names = []

In [6]:
# Function to send a request to the API
def send_request(title, author, fmc_name, retries=3):

    user_prompt = user_prompt_template.format(title=title, author=author, fmc_name = fmc_name)

    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 2000,
        "temperature": 0,
        "response_format": {
            "type": "json_schema",
            "json_schema": { "schema": AnswerFormat.model_json_schema()} },
        "web_search_options": { "search_context_size": "medium" }
        }

    try:
        # Make the request to the API
        response = requests.post(url, json=payload, headers=headers)
        
        # Check if the response is successful
        if response.status_code == 200:
            response_json = response.json()
            answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
            citation = response_json.get("citations", [])
            return answer, citation
        else:
            raise Exception(f"API call failed with status code {response.status_code}")
    
    except Exception as e:
        # Log the error and retry if retries left
        logging.error(f"Error with request for {title} by {author}: {str(e)}")
        if retries > 0:
            logging.info(f"Retrying {retries} more times...")
            sleep(60)  # Sleep to avoid hitting rate limits
            return send_request(title, author, fmc_name, retries - 1)
        else:
            logging.error(f"Failed after retries: {title} by {author}")
            return f"Error: {str(e)}", []

In [7]:
def save_progress():
    if titles:  # Only save if there's something new
        progress_df = pd.DataFrame({
            'title': titles,
            'author': authors,
            'fmc_name': fmc_names,
            'answer': responses,
            'citation': citations
        })

        file_exists = os.path.exists('progress_features.csv')

        if file_exists:
            progress_df.to_csv('progress_features.csv', mode='a', header=False, index=False)
            logging.info(f"Progress appended: {len(titles)} books.")
        else:
            progress_df.to_csv('progress_features.csv', index=False)
            logging.info(f"Progress saved: {len(titles)} books.")

        # Clear the lists so next save only writes new data
        titles.clear()
        authors.clear()
        responses.clear()
        citations.clear()
        fmc_names.clear()

# High 
$10.81
 -> $10.53

$0.28 for 20 books...
/ 10 -> 0.14 
* 1400 = 196.00

# Medium
$10.53
 -> $10.4

$0.13 for 20 books...
/ 10 -> 0.07
* 1400 = 98
## -> medium tut's auch 

In [8]:
start_time = time.time()

# Run for df_subset_1
for index, row in df_sample.iterrows():
    title = row["title"]
    author = row["author"]
    fmc_name = row["fmc_name"]

    # Send request and handle response
    answer, citation = send_request(title, author, fmc_name)
    
    # Store responses
    responses.append(answer)
    citations.append(citation)
    titles.append(title)
    authors.append(author)
    fmc_names.append(fmc_name)

    # Save progress every 10 books
    if (index + 1) % 10 == 0:
        save_progress()

# Save the final progress
save_progress()

# Log the total execution time
end_time = time.time()
execution_time = end_time - start_time
logging.info(f"Completed 100 book requests in {execution_time:.2f} seconds.")

$10.04 (*140) -> $9.23 = 0.81 (*140) -> $113
20 min (*140) -> 2800 min = 46h

In [9]:
df_result = pd.read_csv("progress_features.csv")
df_result

,title,author,fmc_name,answer,citation
0,Dirty Little Secrets JJ Graves Mystery 1,"Hart, Liliana",J.J. Graves,"[[1, 1], [2, 2011], [3, [""mystery"", ""romance"",...",['https://www.goodreads.com/book/show/49587058...
1,The Tycoons Vacation Baby for the Billionaire 2,"Anne, Melody",Trinity Mathews,"[[1, 1], [2, 2011], [3, [""romance"", ""contempor...",['https://www.goodreads.com/book/show/19561986...
2,White Girl Problems,"Walker, Babe",Babe Walker,"[[1, 99, ""Author gender unclear; Babe Walker i...",['https://www.goodreads.com/book/show/12413949...
3,The Empty House,"Pilcher, Rosamunde",Virginia Keile,"[\n[1, 1], 1, ""Rosamunde Pilcher is a female a...",['https://www.goodreads.com/book/show/1054377'...
4,The Unicorn,"Murdoch, Iris",Hannah Crean-Smith,"[[1, 1], [2, 1963], [3, [""gothic fiction"", ""ph...",['https://en.wikipedia.org/wiki/The_Unicorn_(n...
...,...,...,...,...,...
95,Charmed to Death Ophelia Abby 2,"Damsgaard, Shirley",Ophelia Jensen,"[ [1, 1], [-1, -1] , [-1, [""mystery"", ""paranor...",['https://www.goodreads.com/book/show/141036.C...
96,Dealova,"Nuranindya, Dyan",Karra,"[[1, 1], [2, 2005], [3, [""young adult"", ""roman...",['https://en.brilio.net/entertainment/dealova-...
97,The Promise Fallen Star 4,"Sorensen, Jessica",Gemma Lucas,"[[1, 1], [2, 2012], [3, [""young adult"", ""urban...",['https://goodreads.com/book/show/13615578.The...
98,A Boy of Good Breeding,"Toews, Miriam",Knute,"[[1, 1], [2, 2006], [3, [""literary fiction"", ""...",['https://quillandquire.com/review/a-boy-of-go...


In [10]:
for answer in df_result['answer']:
    print(answer)

[[1, 1], [2, 2011], [3, ["mystery", "romance", "suspense", "thriller"]], [4, 99, "No explicit rescue by a male character mentioned in available sources"], [5, 99, "No information about saving a male character"], [6, 1, "Works closely with Detective Jack Lawson, implying professional protection"], [7, 99, "No explicit protection of male characters described"], [8, 99, "Resolution details unclear from available sources"], [9, 1, "Uses her coroner expertise to investigate murders"], [10, 99, "No specific victimization by male characters confirmed"], [11, 99, "No information about harming male characters"], [12, 0, "As coroner, she operates independently in her role"], [13, 1, "Collaborates with law enforcement, implying authority"], [14, 0, "No indications of physical weakness"], [15, 1, "Handles physically demanding coroner duties"], [16, 0, "Actively pursues dangerous investigations"], [17, 1, "Takes risks to solve murders"], [18, 0, "Focuses on investigative work rather than caregiving

In [28]:
len(df_result["answer"].iloc[25])

30

In [15]:
df_result["answer"] = df_result["answer"].apply(ast.literal_eval) # reponses column is string, convert to list

In [17]:
df_result['answer'].apply(lambda x: isinstance(x, (list, tuple)) and len(x) == 29).all()

np.False_

In [24]:
length_check = df_result['answer'].apply(lambda x: hasattr(x, '__len__') and len(x) == 29)
invalid_rows = df_result[~length_check]
print(invalid_rows)

                                                title               author  \
3                                     The Empty House   Pilcher, Rosamunde   
19                            Veronika Decides to Die        Coelho, Paulo   
24                             Ignite Me Shatter Me 3        Mafi, Tahereh   
25                                  The Fourth Estate      Archer, Jeffrey   
30                                 Last Chance Saloon        Keyes, Marian   
37                         Sin City Hot in the City 2     Alexander, Lacey   
48                                 Living Dangerously        Fforde, Katie   
52  The Witches of Santa Anna The Witches of Santa...    Barnholdt, Lauren   
54                                  American Pastoral         Roth, Philip   
56                               Absent in the Spring     Westmacott, Mary   
58                  Arrows Fall Heralds of Valdemar 3     Lackey, Mercedes   
59                    The Golden Crown Tennis Shoes 7  Heimerdin

In [27]:
df_result["answer"].iloc[30]

[[1, 1],
 [-1, 0],
 [2, []],
 [3,
  0,
  "No specific genres were provided in the search results, but based on Marian Keyes' style, it is likely to include genres like chick lit, romance, and comedy."],
 [4, 0, 'There is no indication that Katherine is saved by a male character.'],
 [5, 0, 'There is no indication that Katherine saves a male character.'],
 [6,
  0,
  'There is no indication that Katherine is protected by a male character.'],
 [7, 0, 'There is no indication that Katherine protects a male character.'],
 [8,
  1,
  "Katherine's problems are partly solved through her own efforts and the influence of her friends."],
 [9,
  1,
  'Katherine solves her own problems through her skills and personal growth.'],
 [10,
  0,
  'There is no indication that Katherine is victimized by a male character.'],
 [11, 0, 'There is no indication that Katherine harms a male character.'],
 [12, 0, 'There is no indication that Katherine follows orders.'],
 [13,
  1,
  'As a successful accountant, K

In [ ]:
### Unnest columns 

# Check length of all columns 

# Make all list elements into columns